In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

taxiDB = pd.read_csv('data/taxi_dataset.csv')

In [ ]:
taxiDB.head(5)

<dl>
<dt> Описание колонок:
<dd>id - ID поездки </dd>
<dd>vendor_id - ID компании, осуществляющей перевозку </dd>
<dd>pickup_datetime - Таймкод начала поездки</dd>
<dd>dropoff_datetime - Таймкод конца поездки </dd>
<dd>passenger_count - Количество пассажиров </dd>
<dd>pickup_longitude - Долгота точки, в которой началась поездка </dd>
<dd>pickup_latitude - Широта точки, в которой началась поездка </dd>
<dd>dropoff_longitude - Долгота точки, в которой закончилась поездка </dd>
<dd>dropoff_latitude - Широта точки, в которой закончилась поездка </dd>
<dd>store_and_fwd_flag - Yes/No: Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером </dd>
</dl>

**Целевая переменная - длительность поездки.**

Зная тайм-коды времени начала и конца поездок, можем вычислить обозначенный таргет.
Договоримся, что производим вычисления в секундах.

Положим таргетную переменнул в столбик с названием *trip_duration*

In [ ]:
# приведем pickup_datetime и dropoff_datetime к типу datetime
taxiDB['pickup_datetime'] = pd.to_datetime(taxiDB['pickup_datetime'])
taxiDB['dropoff_datetime'] = pd.to_datetime(taxiDB['dropoff_datetime'])

In [ ]:
# добавим столбец - таргет
taxiDB['trip_duration'] = (taxiDB['dropoff_datetime'] - taxiDB['pickup_datetime']).dt.total_seconds()

In [ ]:
taxiDB.head()

Предсказывая таргет для новых объектов в будущем, мы не будем заранее знать **dropoff_datetime**.

Удалим колонку из датасета.

In [ ]:
taxiDB = taxiDB.drop('dropoff_datetime', axis=1)

Рассмотрим имеющиеся вещественные/бинарные признаки и решим, какие простейшие признаки можно вытащить из остальных колонок.

Во-первых, имеем бинарный признак `vendor_id`, принимающий значения {1, 2}. Переведем его во множество {0, 1}.

In [ ]:
taxiDB['vendor_id'] = taxiDB['vendor_id'] - 1

Также закодируем бинарный признак `store_and_fwd_flag`. 'N' - 0, 'Y' - 1.

In [ ]:
taxiDB['store_and_fwd_flag'] = taxiDB['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)

Во-вторых, можем использовать долготу и широту точек старта/завершения поездки, чтобы примерно оценить расстояние между 2 точками.

Базовая идея состоит в том, чтобы посчитать разность долгот и широт соответственно, то есть:

$$
\delta_{long} = \text{dropoff\_longitude} - \text{pickup\_longitude}
$$

$$
\delta_{lat} =  \text{dropoff\_latitude} - \text{pickup\_latitude}
$$

А потом вычислить географическое расстояние между 2 точками по теореме Пифагора:

$$
R = \sqrt{\delta^2_{long} + \delta^2_{lat}}
$$

Мы реализуем данную задумку и вычислим вещественную колонку *R*.

Только для начала нужно некоторым образом перевести долготу и широту в километры, обеспечив равенство их мер измерения. Потому что, вообще говоря, **градусная мера** широт и долгот имеет неодинаковую шкалу перевода в километры. 

Начнем переводить каждую долготу в некоторое относительно километровое выражение. Соберем список из всех широт и посчитаем медиану

In [ ]:
medianLat = pd.concat([taxiDB['pickup_latitude'], taxiDB['dropoff_latitude']]).median()
medianLat

Теперь, из каждого значения широты вычтем медианное значение.

Результат переведем в километры.

In [ ]:
latMultiplier  = 111.32

taxiDB['pickup_latitude']   = latMultiplier  * (taxiDB['pickup_latitude']   - medianLat)
taxiDB['dropoff_latitude']   = latMultiplier  * (taxiDB['dropoff_latitude']  - medianLat)

Итого, для **latitude** колонок получили следующие выражения:

*На сколько примерно километров севернее или южнее (в зависимости от знака) точка находится относительно средней широты*

In [ ]:
medianLong  = pd.concat([taxiDB['pickup_longitude'], taxiDB['dropoff_longitude']]).median()

longMultiplier = np.cos(medianLat*(np.pi/180.0)) * 111.32

In [ ]:
taxiDB['pickup_longitude']   = longMultiplier  * (taxiDB['pickup_longitude']   - medianLong)
taxiDB['dropoff_longitude']   = longMultiplier  * (taxiDB['dropoff_longitude']  - medianLong)

In [ ]:
taxiDB.head()

Наконец, вычислим географическое расстояние **distance_km**:

In [ ]:
taxiDB['distance_km'] = ((taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude']) ** 2 
+ (taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude']) ** 2) ** (1/2)

In [ ]:
taxiDB.head()

Уберем старые признаки

In [ ]:
taxiDB = taxiDB.drop(['pickup_longitude', 'dropoff_longitude',
                      'pickup_latitude', 'dropoff_latitude'], axis=1)

In [ ]:
taxiDB.head()

Посмотрим, какие значения может принимать `passenger_count`.

In [ ]:
taxiDB['passenger_count'].value_counts()

Количество пассажиров от поездки к поездке ограничено. Вряд ли если к нам придут новые данные, мы увидим числа бОльшие, чем у нас в датасете. Тогда предположим, что данный признак является категориальным

Заменим колонку **passenger_count** колонкой **category_encoded**, используя  **Mean-target encoding**

In [ ]:
taxiDB['category_encoded'] = taxiDB['passenger_count'].map(taxiDB.groupby('passenger_count')['trip_duration'].mean()) 
taxiDB = taxiDB.drop('passenger_count', axis=1)
taxiDB.head()

Остались две колонки: **id**, **pickup_datetime**

**id** можно использовать как обычный идентификатор нашего объекта, поэтому поместите данную колонку в качестве индекса нашей таблички:

In [ ]:
taxiDB = taxiDB.set_index('id')

In [ ]:
taxiDB.head()

Сохраним подготовленные данные

In [ ]:
taxiDB.to_csv('output/processed_data.csv', sep=';')